# Route Simulation

In [1]:
import csv
from typing import List, Dict, Set, Tuple
import glob
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time as ti
import warnings # to suppress warnings
import pickle
from datetime import datetime
warnings.filterwarnings('ignore')

from model import Stop, Edge, StopNetwork, Route, Charger, Bus, Simulation

# Simulations on all buses in Queens Village Depot (QV)

## Load QV Routes

In [2]:
with open("../output/QV_bus_counts.pkl","rb") as f:
    QV_bus_counts = pickle.load(f)

QV_bus_counts["X68"] += 2
print(QV_bus_counts)

{'Q83': 48, 'Q88': 45, 'Q43': 68, 'X63': 7, 'Q36': 40, 'X68': 10, 'X64': 5, 'Q46': 76, 'Q1': 27, 'Q27': 96, 'Q2': 28}


## Initialize stop network

In [3]:
# Initialize the Network
stop_network = StopNetwork()

# Add Routes and Stops into the Network
for route_name in QV_bus_counts.keys():
    stop_network.add_from_csv(route_name, f"../data/routes/{route_name}_stop_graph.csv")
#for filepath in glob.iglob("../data/routes/S*_stop_graph.csv"):
    #route_name = filepath[filepath.rfind("/") + 1:filepath.find("_")]
    #print(filepath)
    #print(route_name)
    #stop_network.add_from_csv(route_name, filepath)

# Set CH as a depot and add chagers to CH depot    
stop_network.stops["Queens Village Depot (QV)"].add_chargers(num_chargers=20, )
stop_network.stops["Queens Village Depot (QV)"].is_depot = True
stop_network.stops["Queens Village Depot (QV)"].charging_capacity=11700
for route in stop_network.routes:
    stop_network.stops["Queens Village Depot (QV)"].Bus_standby.update({route:[]})

#stop_network.add_from_csv("M14D-SBS", "../../data/routes/M14D-SBS_stop_graph.csv")
#stop_network.stops["Michael J. Quill Depot (MQ)"].add_chargers(num_chargers=1)
#stop_network.stops["Michael J. Quill Depot (MQ)"].is_depot = True

## Import KB Bus Schedule

In [4]:
with open("../output/QV_schedule.pkl","rb") as f:
    QV_schedule = pickle.load(f)

stop_network.stops["Queens Village Depot (QV)"].Time_ToBe_Launched = QV_schedule
print(QV_schedule)

{'Q83': ['00:02:00', '01:02:00', '02:02:00', '03:02:00', '04:02:00', '04:22:00', '04:34:00', '04:46:00', '04:58:00', '05:10:00', '05:22:00', '05:32:00', '05:42:00', '05:52:00', '06:00:00', '06:07:00', '06:12:00', '06:17:00', '06:22:00', '06:29:00', '06:32:00', '06:40:00', '06:42:00', '06:50:00', '06:52:00', '06:59:00', '07:01:00', '07:09:00', '07:11:00', '07:19:00', '07:19:00', '07:27:00', '07:27:00', '07:34:00', '07:35:00', '07:42:00', '07:43:00', '07:50:00', '07:51:00', '07:58:00', '08:00:00', '08:08:00', '08:11:00', '08:20:00', '08:23:00', '08:32:00', '08:34:00', '08:43:00', '08:46:00', '08:55:00', '08:58:00', '09:07:00', '09:16:00', '09:25:00', '09:35:00', '09:45:00', '09:55:00', '10:05:00', '10:15:00', '10:25:00', '10:35:00', '10:45:00', '10:55:00', '11:05:00', '11:15:00', '11:25:00', '11:35:00', '11:45:00', '11:55:00', '12:05:00', '12:15:00', '12:25:00', '12:35:00', '12:45:00', '12:57:00', '13:09:00', '13:21:00', '13:33:00', '13:45:00', '13:57:00', '14:09:00', '14:21:00', '14:33:

## Create Buses in the Network

In [5]:
#NUM_BUSES = 1000
queensvillage_depot_routes = list(QV_bus_counts.keys())
#michaeljquill_depot_routes = ['M104']
speed = 15.33 # Km/h
battery_capacity = 400# KWh
battery_charge = 400 * 0.9 # KWh
energy_use_per_km = 1.84 # KWh / Km
#for bus_id in range(NUM_BUSES):
#    route_name = random.choice(michaeljquill_depot_routes)
#    route_direction = random.choice([direction for direction in stop_network.routes[route_name].stops if not "to depot" in direction])
#    cur_stop_name = random.choice([stop.name for stop in stop_network.routes[route_name].stops[route_direction]])
#    stop_network.add_bus(bus_id, speed, cur_stop_name, route_name, route_direction, battery_capacity, battery_charge*(np.random.rand()*(1-0.3)+0.3), energy_use_per_km)
#    print(route_name, route_direction, cur_stop_name)

bus_id = 0
cur_stop_name = "Queens Village Depot (QV)"

for route_name in ["Q83"]:
#for route_name in queensvillage_depot_routes:
    route_direction = stop_network.routes[route_name].get_other_direction(stop_network.stops[cur_stop_name], None)
    for i in range(QV_bus_counts[route_name]):
    #for i in range(33):
        stop_network.add_bus(bus_id, speed, cur_stop_name, route_name, route_direction, battery_capacity, battery_charge, energy_use_per_km)
        bus_id += 1
    print('Created {0:2} buses of Route {1:8} at {2:10} heading {3:10}'.format(i+1, route_name, cur_stop_name, route_direction))


Created 48 buses of Route Q83      at Queens Village Depot (QV) heading CAMBRIA HTS 227 ST via LIBERTY via MRDCK


In [6]:
net_filepath = '../data/Penn_22.csv'
capacity = 100
sim = Simulation(stop_network, net_filepath, capacity)
sim.run(30, 10, '../output/bus_QV.csv','../output/charger_QV.csv','../output/charger_rate_QV.csv')

In [7]:
#stop_network.stops['5 AV/W 38 ST'].edges['to depot of M3'].next_stop
#stop_network.stops["W 231 ST/BROADWAY"].edges
#("to depot of " + route) in test.edges
stop_network.stops["Queens Village Depot (QV)"].Delayed_time

1022540

In [8]:
#for bus in stop_network.routes['M3'].buses:
# print(bus.get_next_direction(stop_network.stops['Michael J. Quill Depot (MQ)'], 'to depot of M3'))
stop_network.stops["Queens Village Depot (QV)"].Delayed_route

['Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',
 'Q46',


In [9]:
#stop_network.stops['W 70 ST/FREEDOM PL'].edges['EAST SIDE YORK AV CROSSTOWN M72'].next_stop
#stop_network.stops['WEST 66 ST/FREEDOM PL'].edges['EAST SIDE YORK AV CROSSTOWN M72'].next_stop
#stop_network.stops['MADISON AV/E 65 ST'].edges
stop_network.routes["Bx3"].stops
#.stops["EAST SIDE YORK AV CROSSTOWN M72"]:
#    print(i)
#stop_network.routes['M72'].get_other_direction(stop_network.stops['W 66 ST/FREEDOM PL'], 'EAST SIDE YORK AV CROSSTOWN M72')

KeyError: 'Bx3'

In [ ]:
#for bus in stop_network.routes['M3'].buses:
#    print(bus.cur_stop.edges['EAST VILLAGE 8 ST via 5 AV'].length)

In [ ]:
#for bus in stop_network.routes['M3'].buses:
#    print(bus.cur_stop.edges['FT GEORGE 193 ST via MADISON via ST NICH'].length)

In [ ]:
#for direction in stop_network.routes['M3'].stops:
#    print(direction)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
 
df = pd.read_csv('../output/charger_rate.csv',sep=',')
df['Time (hr)']=df['Time (s)']/3600
print(df.head())
plt.figure(figsize=(20, 10)) 
display(plt.plot( 'Time (hr)', 'Charger 1', data=df, marker='.',markersize=15, color='pink', linewidth=0))
display(plt.plot( 'Time (hr)', 'Charger 2', data=df, marker='.', color='olive',markersize=7, linewidth=0))
display(plt.plot( 'Time (hr)', 'Charger 3', data=df, marker='.', color='blue', markersize=1, linewidth=0, linestyle='dashed'))

plt.legend()
plt.xlabel('Time (hr)')
plt.ylabel('KWh')
plt.title('Charger Demand')

plt.show()



In [ ]:
import numpy as np
import pandas as pd
 
df = pd.read_csv('../output/charger_rate.csv',sep=',')
plt.figure(figsize=(20, 10)) 
display(plt.plot( 'Time (s)', 'Charger 1', data=df, marker='.',markersize=15, color='pink', linewidth=0))


plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('KWh')
plt.title('Charger 1 Demand')

plt.show()



In [ ]:
test = pd.read_csv('../output/bus.csv',sep=',')
bus_1=test[test["ID"]==1]
bus_2=test[test["ID"]==2]
bus_3=test[test["ID"]==3]

print(bus_1.head(10))

plt.figure(figsize=(20, 10)) 

display(plt.plot('Time (s)', 'Total Distance (Km)', data=bus_1, marker='', color='blue', linewidth=1))
display(plt.plot('Time (s)', 'Total Distance (Km)', data=bus_2, marker='', color='pink', linewidth=1))
display(plt.plot('Time (s)', 'Total Distance (Km)', data=bus_3, marker='', color='red', linewidth=1))

plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Distance km')
plt.title('Test Bus')


In [ ]:
test = pd.read_csv('../output/bus.csv',sep=',')

test['Time (hr)']=test['Time (s)']/3600
print(df.head())

bus_1=test[test["ID"]==1]
bus_2=test[test["ID"]==2]
bus_3=test[test["ID"]==3]


#print(bus_1.head(10))

plt.figure(figsize=(20, 10)) 

display(plt.plot('Time (hr)', 'SOC', data=bus_1, marker='', color='blue', linewidth=1))
display(plt.plot('Time (hr)', 'SOC', data=bus_2, marker='', color='red', linewidth=1))
display(plt.plot('Time (hr)', 'SOC', data=bus_3, marker='', color='green', linewidth=1))


plt.legend()
plt.xlabel('Time (hr)')
plt.ylabel('SOC')
plt.title('Test Bus')


In [ ]:
host_cap = pd.read_csv('../data/Penn_22.csv',sep=',',index_col=None)
dff = host_cap[['DateTime','Pennsylvania']]
dff['DateTime']= pd.to_datetime(dff['DateTime'])
dff['Hour'] = dff.DateTime.apply(lambda x: x.hour)

In [ ]:
#12 chargers @450kW 
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', 
          '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
charger = [5.4, 5.4, 5.4, 5.4, 5.4, 5.4, 5.4, 5.4, 5.4, 0, 0, 0, 0, 0, 0, 0, 0,
          5.4, 5.4, 5.4, 5.4, 5.4, 5.4, 5.4]
load = []
for i in range(0,24):
    ld = dff['Pennsylvania'].loc[5352+i]
    load.append(float(ld))
avg_load = []
for i in range(0,24):
    avg = dff.loc[dff['Hour'] == i, 'Pennsylvania'].mean()
    avg = "%.2f" %avg
    avg_load.append(float(avg))
width = 0.7

fig, ax = plt.subplots(figsize=(30,15))
plt.axhline(y=100, color='r', linestyle='--', label='Capability 100MW')
plt.plot(labels, avg_load, marker='o', markerfacecolor='black', markersize=10, color='skyblue', linewidth=4, label='Average Anticipated Load')
ax1 = ax.bar(labels, load, width, color='green', label='Anticipated Load')
ax2 = ax.bar(labels, charger, width, color='blue', yerr=None, bottom=load, label='Bus Charger Load')
for r1, r2 in zip(ax1, ax2):
    h1 = r1.get_height()
    h2 = r2.get_height()
    plt.text(r1.get_x() + r1.get_width() / 2., h1 / 2., "%.2f" % h1, ha="center", va="center", color="white", fontsize=16)
    plt.text(r2.get_x() + r2.get_width() / 2., h1 + h2 / 2., "%.2f" % h2, ha="center", va="center", color="white", fontsize=16)
ax.set_xlabel('Hour')
ax.set_ylabel('MW')
ax.set_title('Pennsylvania Network Hosting Capacity at Aug 12, 2022')
ax.legend(loc='best')
plt.ylim(0,300)
plt.show()

In [ ]:
#26 chargers @450kW 
charger = [11.7, 11.7, 11.7, 11.7, 11.7, 11.7, 11.7, 11.7, 11, 0, 0, 0, 0, 0, 0, 0, 0,
          9, 11.7, 11.7, 11.7, 11.7, 11.7, 11.7]
load = []
for i in range(0,24):
    ld = dff['Pennsylvania'].loc[5352+i]
    load.append(float(ld))
avg_load = []
for i in range(0,24):
    avg = dff.loc[dff['Hour'] == i, 'Pennsylvania'].mean()
    avg = "%.2f" %avg
    avg_load.append(float(avg))
width = 0.7

fig, ax = plt.subplots(figsize=(30,15))
plt.axhline(y=100, color='r', linestyle='--', label='Capability 100MW')
plt.plot(labels, avg_load, marker='o', markerfacecolor='black', markersize=10, color='skyblue', linewidth=4, label='Average Anticipated Load')
ax1 = ax.bar(labels, load, width, color='green', label='Anticipated Load')
ax2 = ax.bar(labels, charger, width, color='blue', yerr=None, bottom=load, label='Bus Charger Load')
for r1, r2 in zip(ax1, ax2):
    h1 = r1.get_height()
    h2 = r2.get_height()
    plt.text(r1.get_x() + r1.get_width() / 2., h1 / 2., "%.2f" % h1, ha="center", va="center", color="white", fontsize=16)
    plt.text(r2.get_x() + r2.get_width() / 2., h1 + h2 / 2., "%.2f" % h2, ha="center", va="center", color="white", fontsize=16)
ax.set_xlabel('Hour')
ax.set_ylabel('MW')
ax.set_title('Pennsylvania Network Hosting Capacity at Aug 12, 2022')
ax.legend(loc='best')
plt.ylim(0,300)
plt.show()

In [ ]:
test_sim4c = pd.DataFrame(columns = ('origin', 'destination', 'route_name', 'direction', 'distance'))

with open("../data/routes/SIM4C_stop_graph.csv") as csv_file:
            reader = csv.reader(csv_file)
            destinations = next(reader)[1:]
            for row in reader:
                origin = row[0]
                for destination, distance_direction in zip(destinations, row[1:]):
                    if distance_direction != "":
                        distance, direction = distance_direction.split(" ", 1)
                        distance = float(distance)
                        test_sim4c = test_sim4c.append({'origin':origin, 'destination':destination, 'route_name':route_name, 'direction':direction, 'distance':distance},ignore_index=True)
                    
           

In [ ]:
test_sim4c